In [12]:
import requests
import pandas as pd
from datetime import datetime
import json
import time
from google.colab import files  # For uploading files in Colab
import http.client

# Function to check if a website is working properly
def is_website_working(url):
    try:
        # Send a GET request to the website
        response = requests.get(url, timeout=10)

        # Check if the response status code is 404 (not found)
        if response.status_code == 404:
            print(f"Website {url} is not found (404).")
            return False

        # Check if the page contains a CAPTCHA (e.g., by looking for "captcha" in the content)
        if "captcha" in response.text.lower():
            print(f"Website {url} is asking for CAPTCHA.")
            return False

        # Check if the status code is 200 (OK)
        if response.status_code == 200:
            return True

    except requests.RequestException as e:
        print(f"Error accessing {url}: {e}")

    return False

# Upload the Excel file
uploaded = files.upload()

# Load the city, state, and country data from the uploaded Excel file
excel_filename = list(uploaded.keys())[0]  # This assumes you upload one file
sheet_name = "Sheet1"  # Change this if your sheet has a different name

# Read the city, state, and country data from the Excel file
cities_df = pd.read_excel(excel_filename, sheet_name=sheet_name)

# Get the current date and time for the output file name
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
csv_filename = f"/content/result{current_time}.xlsx"  # The output file name will be resultYYYYMMDD_HHMMSS.xlsx

# Store previous yoga centers to check for duplicates across rows (tracking by placeId)
previous_centers = {}

# Create an Excel writer to write the results into an Excel file
with pd.ExcelWriter(csv_filename, engine='openpyxl') as writer:  # Change the engine to 'openpyxl'
    # Create a list to store all the rows for final writing to Excel
    all_rows = []

    # Write headers for city, state, country, and up to 20 yoga centers
    header = ["City", "State", "Country"]
    for i in range(1, 21):  # Adjusted for 20 centers
        header.extend([f"center{i}name", f"center{i}website", f"center{i}review"])

    # Iterate through each row in the Excel data
    for index, row in cities_df.iterrows():
        total_rows = len(cities_df)  # Total number of rows in the sheet
        print(f"Processing row {index + 1}/{total_rows}...")  # Show the current row number and total rows

        city = row['city']
        state = row['state']
        country = row['country']

        # Construct the search query for yoga studios
        query = f"yoga in {city}, {state}, {country}"

        # Initialize row data with city, state, country
        row_data = [city, state, country]

        # Keep track of seen center IDs to avoid duplicates in this row
        seen_center_ids = set()

        # Try the first search (city, state, country)
        response_json = None
        try:
            # API Connection
            conn = http.client.HTTPSConnection("google.serper.dev")
            payload = json.dumps({
                "q": query,
                "gl": "AR"  # Modify location as needed
            })
            headers = {
                'X-API-KEY': 'abfe315b7f9e4392041a4aff90c4ccd4ebcdfad4',  # Use your valid API key
                'Content-Type': 'application/json'
            }

            # Send request
            conn.request("POST", "/maps", payload, headers)
            res = conn.getresponse()
            data = res.read()
            response_json = json.loads(data)

            if 'statusCode' in response_json and response_json['statusCode'] == 403:
                print(f"Response from first search for {query}: {response_json}")
                raise Exception(f"Unauthorized access for {query}")

        except Exception as e:
            print(f"Error in first search for {query}: {e}")

        # If no results from the first search, try searching just by country
        if response_json is None or 'places' not in response_json or len(response_json['places']) == 0:
            print(f"Searching for yoga in {country}...")
            query = f"yoga in {country}"
            try:
                # Send request for country search
                conn.request("POST", "/maps", payload, headers)
                res = conn.getresponse()
                data = res.read()
                response_json = json.loads(data)

                if 'statusCode' in response_json and response_json['statusCode'] == 403:
                    print(f"Response from second search for {query}: {response_json}")
                    raise Exception(f"Unauthorized access for {query}")

            except Exception as e:
                print(f"Error in second search for {query}: {e}")

        # If still no valid response, continue to next row
        if 'places' not in response_json or len(response_json['places']) == 0:
            print(f"No yoga centers found for {city}, {state}, {country}")
            row_data.extend(["N/A", "N/A", "N/A"] * 20)  # Fill with "N/A" if no results found
            all_rows.append(row_data)
            continue

        # Extract the details for each yoga studio in the response
        centers_found = 0
        print(f"Searching for centers in {city}, {state}, {country}...")
        for place in response_json.get("places", []):
            name = place.get("title", "N/A")
            website = place.get("website", "N/A")
            place_id = place.get("placeId", None)

            # Skip the place if it already exists in the same row (check by placeId)
            if place_id and place_id in seen_center_ids:
                continue
            # Add to seen centers set to track uniqueness in this row and globally across previous rows
            if place_id:
                seen_center_ids.add(place_id)
                previous_centers[place_id] = {"name": name, "website": website, "review_url": f"https://www.google.com/maps/place/?q=place_id:{place_id}"}

            # Only check the website if it's valid
            if website != "N/A" and is_website_working(website):
                # Construct the review URL using place_id if available
                review_url = f"https://www.google.com/maps/place/?q=place_id:{place_id}" if place_id else "N/A"

                # Add center details if the website is working
                print(f"Found working website: {name}, {website}, {review_url}")
                row_data.extend([name, website, review_url])
                centers_found += 1

            # Stop adding centers after the first 20 valid ones
            if centers_found >= 20:
                break

        # Fill in the missing center details with "N/A" if there are less than 20 valid centers
        while centers_found < 20:
            row_data.extend(["N/A", "N/A", "N/A"])
            centers_found += 1

        # Append row data to the list of all rows
        all_rows.append(row_data)

        # Be respectful of rate limits by pausing between requests if needed
        time.sleep(1)  # Adjust sleep time based on API rate limits

# Write all the data into the Excel file
df = pd.DataFrame(all_rows, columns=header)
df.to_excel(writer, index=False, sheet_name='Yoga Centers')

print(f"Scraped results from {len(cities_df)} cities and saved to {csv_filename}")

# Use the following command to download the file to your local machine
files.download(csv_filename)


Scraped results from 497 cities and saved to /content/result20250219_163144.xlsx
